In [38]:
import json
import requests
import pandas as pd
import gc

# *.json file with API key for alphavantage
with open('../config/config.json') as f:
    config = json.load(f)


In [39]:
# Function to derive the stock data for a specific stock and transform it into pandas dataframe
def get_stock_data(symbol : str, api_key : str):
    endpoint = "https://www.alphavantage.co/query"

    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': symbol,
        'outputsize': 'full',
        'datatype': 'json',
        'apikey': api_key
    }

    response = requests.get(endpoint, params)

    if response.status_code == 200:
        data = response.json()
        if 'Time Series (Daily)' in data:
            time_series = data['Time Series (Daily)']
            
            pandasDf = pd.DataFrame.from_dict(time_series, orient='index')
            pandasDf.index = pd.to_datetime(pandasDf.index)
            pandasDf = pandasDf[['4. close']]
            pandasDf.rename(columns={'4. close': 'Close'}, inplace=True)
            return pandasDf
    else:
        return None


In [40]:
# Download and save as csv the TSLA stock data and NIO stock data
dataTSLA =  get_stock_data('TSLA', config['ALPHA_VANTAGE_API_KEY'])
dataNIO = get_stock_data('NIO', config['ALPHA_VANTAGE_API_KEY'])

dataTSLA.to_csv('../data/TSLA.csv')
dataNIO.to_csv('../data/NIO.csv')

In [41]:
dataTSLA = dataTSLA.loc[dataNIO.index]
display(dataTSLA.head())
display(dataNIO.head())

,Close
2025-01-06,411.0500
2025-01-03,410.4400
2025-01-02,379.2800
2024-12-31,403.8400
2024-12-30,417.4100


,Close
2025-01-06,4.7100
2025-01-03,4.6300
2025-01-02,4.5500
2024-12-31,4.3600
2024-12-30,4.3800


In [42]:
# Combine the datasets for TSLA and NIO
dataTSLA.rename(columns={'Close': 'CloseTSLA'}, inplace=True)
dataTSLA['CloseNIO'] = dataNIO['Close']

finalData = dataTSLA
del dataTSLA, dataNIO
gc.collect()

display(finalData.head())

,CloseTSLA,CloseNIO
2025-01-06,411.0500,4.7100
2025-01-03,410.4400,4.6300
2025-01-02,379.2800,4.5500
2024-12-31,403.8400,4.3600
2024-12-30,417.4100,4.3800


In [43]:
# Set the close values tobe numeric and not strings
finalData['CloseTSLA'] = pd.to_numeric(finalData['CloseTSLA'], errors='coerce')
finalData['CloseNIO'] = pd.to_numeric(finalData['CloseNIO'], errors='coerce')

In [44]:
# Calculate spread and z score
finalData['spread'] = finalData['CloseTSLA'] - finalData['CloseNIO']
finalData['zscore'] = (finalData['spread'] - finalData['spread'].mean()) / finalData['spread'].std()

finalData.to_csv('../data/data_Spread_Z.csv')

display(finalData.head())

,CloseTSLA,CloseNIO,spread,zscore
2025-01-06,411.05,4.71,406.34,-0.175904
2025-01-03,410.44,4.63,405.81,-0.177521
2025-01-02,379.28,4.55,374.73,-0.272347
2024-12-31,403.84,4.36,399.48,-0.196834
2024-12-30,417.41,4.38,413.03,-0.155493


In [45]:
# Function to generate basic signals based on z score
def generate_signals(data : pd.DataFrame, z_threshold = 1.5):
    data['signal'] = 0
    data.loc[data['zscore'] > z_threshold, 'signal'] = -1  # Short spread
    data.loc[data['zscore'] < -z_threshold, 'signal'] = 1  # Long spread
    data.loc[data['zscore'].abs() < 0.5, 'signal'] = 0

    return data

In [46]:
# Function to simulate trades
def simulate_trades(data, initial_cash=100000):
    cash = initial_cash
    position = 0
    for idx, row in data.iterrows():
        if row['signal'] == 1:  # Long
            position += 1
            cash -= row['spread']
        elif row['signal'] == -1:  # Short
            position -= 1
            cash += row['spread']
        elif row['signal'] == 0:  # Close
            cash += position * row['spread']
            position = 0
    return cash

In [47]:
data = generate_signals(finalData)

display(data[['zscore', 'signal']].head())

,zscore,signal
2025-01-06,-0.175904,0
2025-01-03,-0.177521,0
2025-01-02,-0.272347,0
2024-12-31,-0.196834,0
2024-12-30,-0.155493,0


In [48]:
final_cash = simulate_trades(data)

print(final_cash)

139049.45000000004
